In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
q_data = pd.read_csv('2019-2020_school_year/pdets.csv')
data_response = pd.read_csv('2019-2020_school_year/plogs.csv')

In [3]:
data_response.head()

,log_id,student_id,assignment_id,problem_id,start_time,time_on_task,answer_before_tutoring,fraction_of_hints_used,attempt_count,answer_given,problem_completed,correct
0,139358,1092,13941,1497672,2019-09-02 12:21:21.910000-04:00,89.299,True,NaN,2,False,True,False
1,139358,1092,13941,1497673,2019-09-02 12:22:52.264000-04:00,44.049,True,NaN,3,True,True,False
2,139358,1092,13941,1497674,2019-09-02 12:23:37.463000-04:00,22.290,True,NaN,1,False,True,True
3,139358,1092,13941,1497675,2019-09-02 12:24:00.948000-04:00,290.005,True,NaN,2,True,True,False
4,139358,1092,13941,1497676,2019-09-03 21:46:04.396000-04:00,34.116,True,NaN,1,False,True,True


In [4]:
q_data.head()

,problem_id,content_source,skills,problem_type,tutoring_types,student_answer_count,mean_correct,mean_time_on_task
0,16.0,"['Certified Content', 'Engage New York']",['8.F.B.5'],Multiple Choice,['Scaffold'],16,0.875000,62.389875
1,33.0,"['Certified Content', 'Engage New York']",['8.NS.A.2-1'],Multiple Choice,['Hint'],80,0.734177,NaN
2,35.0,"['Certified Content', 'Engage New York']",['8.NS.A.2-1'],Exact Match (case sensitive),['Hint'],34,0.323529,NaN
3,37.0,"['Certified Content', 'Engage New York']",['8.NS.A.2-1'],Exact Match (case sensitive),['Hint'],28,0.857143,NaN
4,39.0,"['Certified Content', 'Engage New York']",['8.NS.A.2-1'],Multiple Choice,['Hint'],26,0.423077,NaN


In [5]:
data_response = data_response.dropna(subset=['correct'])
q_data = q_data.dropna(subset=['skills'])
data_response = data_response.loc[data_response['problem_id'].isin(q_data['problem_id'].unique())]

In [6]:
response_data = dict()
stu_response_data = dict() 
stu_num = np.random.choice(np.arange(163097), size=2000, replace=False)
least_respone_num = 40
original_stu_map = dict()
original_cnt_stu = 0

for stu in data_response["student_id"].unique():
    original_stu_map[original_cnt_stu] = stu
    original_cnt_stu += 1

In [7]:
for stu in tqdm(stu_num, desc='Filter student'):
    stu_data = data_response.loc[data_response["student_id"] == original_stu_map[stu]]
    for data in stu_data.values:
        tmp_data = (stu, data[3])
        response_data[tmp_data] = (data[11] == True)

for key, value in response_data.items():
    if key[0] not in stu_response_data:
        stu_response_data[key[0]] = []
    stu_response_data[key[0]].append([int(key[0]), key[1], int(value)])

Filter student:   0%|          | 0/2000 [00:00<?, ?it/s]

Filter student: 100%|██████████| 2000/2000 [00:10<00:00, 189.82it/s]


In [8]:
stu_map = dict()
cnt_stu = 0
question_set = set()
cnt_question = 0
question_map = dict()
concept_set = set()
cnt_concept = 0
concept_map = dict()


for key, value in tqdm(stu_response_data.items(), desc='Remap student_id, question_id and concept_id'):
    if len(value) >= least_respone_num:
        stu_map[key] = cnt_stu
        cnt_stu += 1
        for data in value:
            question_set.add(data[1])
            for concept in q_data.loc[q_data["problem_id"] == data[1]]['skills'].iloc[0].split(','):
                concept_set.add(concept)


for question in question_set:
    question_map[question] = cnt_question
    cnt_question += 1

for concept in concept_set:
    concept_map[concept] = cnt_concept
    cnt_concept += 1

Remap student_id, question_id and concept_id: 100%|██████████| 2000/2000 [00:15<00:00, 132.96it/s]


In [9]:
TotalData = []
q_matrix = np.zeros((cnt_question, cnt_concept))

for key, value in tqdm(stu_response_data.items(), desc='Construct final data'):
    if len(value) >= least_respone_num:
        for data in value:
            TotalData.append([stu_map[data[0]], question_map[data[1]], data[2]])
            for concept in q_data.loc[q_data["problem_id"] == data[1]]['skills'].iloc[0].split(','):
                q_matrix[question_map[data[1]]][concept_map[concept]] = 1

print('Final student number: {}, Final question number: {}, Final concept number: {}, Final response number: {}'.format(cnt_stu, cnt_question, cnt_concept, len(TotalData)))

Construct final data: 100%|██████████| 2000/2000 [00:14<00:00, 142.34it/s]


Final student number: 416, Final question number: 13234, Final concept number: 646, Final response number: 43605


In [10]:
# 提取第三列
third_column = [row[2] for row in TotalData]

# 计算平均值
average = np.mean(third_column)

In [11]:
average

0.6872377020983832